<a href="https://colab.research.google.com/github/suellensr/SantanderBootcamp_DIO/blob/main/Projeto_DIO_pipeline_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O objetivo é cruzar o arquivo de transações do Santander (dado interno no Excel - que aqui foram apenas dados aleatórios, não refletindo a situação real) com os valores das cotações das ações do Santander na bolsa, realizando o pipeline ETL necessário, para possibilitar futuras análises de correlação de performance do Santander e alta no mercado.

In [135]:
# Importando as bibliotecas necessárias
import yfinance as yf
import pandas as pd
from pandas.core.resample import date_range

In [136]:
# Puxar dados do yahoofinance em um període de 90 dias com intervalo diário
symbol = 'SAN'
santander = yf.Ticker(symbol)
historical_data = santander.history(period="90d",interval="1d")

In [137]:
historical_data = santander.history(period="90d",interval="1d")

In [138]:
# Criar um dataframe com os dados puxados do yahoofinance, resetando o index, para criar a coluna date
df = pd.DataFrame(historical_data).reset_index()
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-06-05 00:00:00-04:00,3.40,3.41,3.36,3.40,3073300,0.0,0.0
1,2023-06-06 00:00:00-04:00,3.35,3.45,3.34,3.44,3227300,0.0,0.0
2,2023-06-07 00:00:00-04:00,3.41,3.44,3.40,3.44,2594800,0.0,0.0
3,2023-06-08 00:00:00-04:00,3.45,3.48,3.42,3.48,2930700,0.0,0.0
4,2023-06-09 00:00:00-04:00,3.42,3.43,3.41,3.43,1459900,0.0,0.0


In [139]:
# Conferindo se há dados faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                           
---  ------        --------------  -----                           
 0   Date          90 non-null     datetime64[ns, America/New_York]
 1   Open          90 non-null     float64                         
 2   High          90 non-null     float64                         
 3   Low           90 non-null     float64                         
 4   Close         90 non-null     float64                         
 5   Volume        90 non-null     int64                           
 6   Dividends     90 non-null     float64                         
 7   Stock Splits  90 non-null     float64                         
dtypes: datetime64[ns, America/New_York](1), float64(6), int64(1)
memory usage: 5.8 KB


In [140]:
# Tratando e transformando a coluna date (tirando a hora) para ser passível do merge com aquivo Excel

df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.date

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          90 non-null     object 
 1   Open          90 non-null     float64
 2   High          90 non-null     float64
 3   Low           90 non-null     float64
 4   Close         90 non-null     float64
 5   Volume        90 non-null     int64  
 6   Dividends     90 non-null     float64
 7   Stock Splits  90 non-null     float64
dtypes: float64(6), int64(1), object(1)
memory usage: 5.8+ KB


In [141]:
# Definindo o tipo de Date de object para datetime64
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          90 non-null     datetime64[ns]
 1   Open          90 non-null     float64       
 2   High          90 non-null     float64       
 3   Low           90 non-null     float64       
 4   Close         90 non-null     float64       
 5   Volume        90 non-null     int64         
 6   Dividends     90 non-null     float64       
 7   Stock Splits  90 non-null     float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 5.8 KB


In [142]:
# DROP das colunas não necessárias
df2 = df.drop(columns=['Open', 'High','Low','Volume', 'Dividends','Stock Splits'])
df2.head()

,Date,Close
0,2023-06-05,3.40
1,2023-06-06,3.44
2,2023-06-07,3.44
3,2023-06-08,3.48
4,2023-06-09,3.43


In [143]:
# Criar um dataframe com os dados do arquivo excel
arquivo = '/content/Dados_Santander.xlsx'
df3 = pd.read_excel(arquivo)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         101 non-null    datetime64[ns]
 1   Transaction  101 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.7 KB


In [144]:
#Identificando os finais de semana para DROPAR. Sabado (5) e Domingo (6).
#A bolsa não funciona finais de semana

df3['diadasemana'] = df3['Date'].dt.dayofweek
df3 = df3[(df3['diadasemana'] != 5) & (df3['diadasemana'] != 6)]
df3.head(15)

,Date,Transaction,diadasemana
0,2023-07-03,71,0
1,2023-07-04,39,1
2,2023-07-05,65,2
3,2023-07-06,86,3
4,2023-07-07,18,4
7,2023-07-10,12,0
8,2023-07-11,49,1
9,2023-07-12,78,2
10,2023-07-13,87,3
11,2023-07-14,28,4


In [145]:
# Criar um dataframe com a junção dos dataframes anteriores
merged_df = df2.merge(df3, on='Date', how='inner')
merged_df.head()

,Date,Close,Transaction,diadasemana
0,2023-07-03,3.75,71,0
1,2023-07-05,3.63,65,2
2,2023-07-06,3.53,86,3
3,2023-07-07,3.58,18,4
4,2023-07-10,3.59,12,0


In [146]:
# Fazer um DROP da coluna diadasemana
merged_df2 = merged_df.drop(columns=['diadasemana'])
merged_df2.head()


,Date,Close,Transaction
0,2023-07-03,3.75,71
1,2023-07-05,3.63,65
2,2023-07-06,3.53,86
3,2023-07-07,3.58,18
4,2023-07-10,3.59,12


In [148]:
# Fazendo um LOAD dos dados em um arquivo excel
caminho = '/content/Dados_Santander_Acoes.xlsx'
merged_df2.to_excel(caminho, index=False)